# 5장. 회원 탈퇴를 예측하는 테크닉 10

In [1]:
import warnings
warnings.filterwarnings('ignore')

## 41: 데이터 읽어들이고 이용 데이터 수정

데이터 읽어들이기

In [2]:
import pandas as pd
customer = pd.read_csv('customer_join.csv')
uselog_months = pd.read_csv('use_log_months.csv')

In [3]:
customer.head()

,customer_id,name,class,gender,start_date,end_date,campaign_id,is_deleted,class_name,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period
0,OA832399,XXXX,C01,F,2015-05-01,NaN,CA1,0,종일,10500,일반,4.833333,5.0,8,2,1,2019-04-30,47
1,PL270116,XXXXX,C01,M,2015-05-01,NaN,CA1,0,종일,10500,일반,5.083333,5.0,7,3,1,2019-04-30,47
2,OA974876,XXXXX,C01,M,2015-05-01,NaN,CA1,0,종일,10500,일반,4.583333,5.0,6,3,1,2019-04-30,47
3,HD024127,XXXXX,C01,F,2015-05-01,NaN,CA1,0,종일,10500,일반,4.833333,4.5,7,2,1,2019-04-30,47
4,HD661448,XXXXX,C03,F,2015-05-01,NaN,CA1,0,야간,6000,일반,3.916667,4.0,6,1,1,2019-04-30,47


In [4]:
uselog_months.head()

,연월,customer_id,count
0,201804,AS002855,4
1,201804,AS009013,2
2,201804,AS009373,3
3,201804,AS015315,6
4,201804,AS015739,7


데이터 가공
- 미래를 예측하기 위해 그달과 1개월 전의 이용 이력만으로 데이터 작성

In [5]:
year_months = list(uselog_months["연월"].unique())
print(year_months)
uselog = pd.DataFrame()
for i in range(1, len(year_months)):
    tmp = uselog_months.loc[uselog_months["연월"]==year_months[i]]
    tmp.rename(columns={"count":"count_0"}, inplace=True)
    tmp_before = uselog_months.loc[uselog_months["연월"]==year_months[i-1]]
    del tmp_before["연월"]
    tmp_before.rename(columns={"count":"count_1"}, inplace=True)
    tmp = pd.merge(tmp, tmp_before, on="customer_id", how="left")
    uselog = pd.concat([uselog, tmp], ignore_index=True)
uselog.head()

[201804, 201805, 201806, 201807, 201808, 201809, 201810, 201811, 201812, 201901, 201902, 201903]


,연월,customer_id,count_0,count_1
0,201805,AS002855,5,4.0
1,201805,AS009373,4,3.0
2,201805,AS015233,7,NaN
3,201805,AS015315,3,6.0
4,201805,AS015739,5,7.0


## 42: 탈퇴전월의 탈퇴고객데이터 작성

- 월말까지 탈퇴 신청을 해야 다음 달 말에 탈퇴 가능
- 2018.08: 탈퇴 신청, 탈퇴 전월
- 2018.09: 탈퇴 신청 완료, 탈퇴월
- 2019.10: 탈퇴
- 따라서 탈퇴월을 8월로 설정, 그리고 그 전 달인 7월의 데이터로부터 8월에 탈퇴 신청 할 확률 예측

In [6]:
from dateutil.relativedelta import relativedelta
exit_customer = customer.loc[customer["is_deleted"]==1]
exit_customer["exit_date"] = None # 초기값 none; 나중에 최종 탈퇴 날짜 저장
exit_customer["end_date"] = pd.to_datetime(exit_customer["end_date"])
for i in range(len(exit_customer)):
    exit_customer["exit_date"].iloc[i] = exit_customer["end_date"].iloc[i] - relativedelta(months=1)
exit_customer["연월"] = pd.to_datetime(exit_customer["exit_date"]).dt.strftime("%Y%m")
uselog["연월"] = uselog["연월"].astype(str)
exit_uselog = pd.merge(uselog, exit_customer, on=["customer_id", "연월"], how="left")
print(len(uselog))
exit_uselog.head()

33851


,연월,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
0,201805,AS002855,5,4.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,201805,AS009373,4,3.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,201805,AS015233,7,NaN,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,201805,AS015315,3,6.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,201805,AS015739,5,7.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


- .loc: 라벨(Label; 데이터프레임의 인덱스/열 이름)을 기준으로 데이터 선택
- .loc: 정수 위치를 기준으로 데이터 선택

결측치 없는 데이터만 남기기

In [7]:
exit_uselog = exit_uselog.dropna(subset=["name"])
print(len(exit_uselog))
print(len(exit_uselog["customer_id"].unique()))
exit_uselog.head()

1104
1104


,연월,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
19,201805,AS055680,3,3.0,XXXXX,C01,M,2018-03-01,2018-06-30,CA1,...,10500.0,일반,3.000000,3.0,3.0,3.0,0.0,2018-06-30,3.0,2018-05-30 00:00:00
57,201805,AS169823,2,3.0,XX,C01,M,2017-11-01,2018-06-30,CA1,...,10500.0,일반,3.000000,3.0,4.0,2.0,1.0,2018-06-30,7.0,2018-05-30 00:00:00
110,201805,AS305860,5,3.0,XXXX,C01,M,2017-06-01,2018-06-30,CA1,...,10500.0,일반,3.333333,3.0,5.0,2.0,0.0,2018-06-30,12.0,2018-05-30 00:00:00
128,201805,AS363699,5,3.0,XXXXX,C01,M,2018-02-01,2018-06-30,CA1,...,10500.0,일반,3.333333,3.0,5.0,2.0,0.0,2018-06-30,4.0,2018-05-30 00:00:00
147,201805,AS417696,1,4.0,XX,C03,F,2017-09-01,2018-06-30,CA1,...,6000.0,일반,2.000000,1.0,4.0,1.0,0.0,2018-06-30,9.0,2018-05-30 00:00:00


## 43: 지속 회원의 데이터 작성

- 지속 회원은 탈퇴 월이 없음; 어떤 연월이든 ok
- 지속 회원 추출 후 uselog 데이터에 결합

In [8]:
conti_customer = customer.loc[customer["is_deleted"]==0]
conti_uselog = pd.merge(uselog, conti_customer, on=["customer_id"], how="left")
print(len(conti_uselog))
conti_uselog = conti_uselog.dropna(subset=["name"]) #name 칼럼의 결손 데이터 제거, 탈퇴 회원 제거
print(len(conti_uselog))

33851
27422


- 지속회원 데이터는 지금 불균형한 데이터
-> 지속 회원 데이터를 회원당 1개가 되게 언더샘플링

데이터를 섞고 중복을 제거
- sample(frac=1): 데이터프레임의 모든 행을 무작위로 섞음
    - frac=1: 데이터프레임의 100%를 샘플링한다는 뜻
- reset_indes(drop=True): 데이터프레임의 인덱스를 다시 설정
    - drop=True: 이전의 인덱스를 삭제하고, 새로운 순차적 인덱스 부여
- drop_duplicates: 데이터프레임에서 중복된 행을 제거하는 메서드

In [9]:
conti_uselog = conti_uselog.sample(frac=1).reset_index(drop=True)
conti_uselog = conti_uselog.drop_duplicates(subset="customer_id")
print(len(conti_uselog))
conti_uselog.head()

2842


,연월,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,class_name,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period
0,201903,PL666665,7,3.0,XXX,C01,M,2016-12-01,NaN,CA1,...,종일,10500.0,일반,5.416667,5.0,9.0,3.0,1.0,2019-04-30,28.0
1,201903,IK287071,3,8.0,XXXXX,C02,F,2017-04-01,NaN,CA1,...,주간,7500.0,일반,6.333333,7.0,9.0,3.0,1.0,2019-04-30,24.0
2,201902,HI754520,3,5.0,XXXXX,C01,F,2016-04-01,NaN,CA1,...,종일,10500.0,일반,4.083333,4.0,6.0,3.0,1.0,2019-04-30,36.0
3,201812,PL459045,2,5.0,XXXX,C01,M,2016-05-01,NaN,CA1,...,종일,10500.0,일반,5.000000,5.5,8.0,2.0,1.0,2019-04-30,35.0
4,201810,AS902294,7,5.0,XXXXXX,C02,F,2017-07-01,NaN,CA2,...,주간,7500.0,입회비반액할인,5.833333,6.5,7.0,2.0,1.0,2019-04-30,21.0


지속 회원 데이터와 탈퇴 회원 데이터 결합 (세로)

In [10]:
predict_data = pd.concat([conti_uselog, exit_uselog],ignore_index=True)
print(len(predict_data))
predict_data.head()

3946


,연월,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
0,201903,PL666665,7,3.0,XXX,C01,M,2016-12-01,NaT,CA1,...,10500.0,일반,5.416667,5.0,9.0,3.0,1.0,2019-04-30,28.0,NaN
1,201903,IK287071,3,8.0,XXXXX,C02,F,2017-04-01,NaT,CA1,...,7500.0,일반,6.333333,7.0,9.0,3.0,1.0,2019-04-30,24.0,NaN
2,201902,HI754520,3,5.0,XXXXX,C01,F,2016-04-01,NaT,CA1,...,10500.0,일반,4.083333,4.0,6.0,3.0,1.0,2019-04-30,36.0,NaN
3,201812,PL459045,2,5.0,XXXX,C01,M,2016-05-01,NaT,CA1,...,10500.0,일반,5.000000,5.5,8.0,2.0,1.0,2019-04-30,35.0,NaN
4,201810,AS902294,7,5.0,XXXXXX,C02,F,2017-07-01,NaT,CA2,...,7500.0,입회비반액할인,5.833333,6.5,7.0,2.0,1.0,2019-04-30,21.0,NaN


## 44: 예측할 달의 재적 기간 작성

재적 기간(period) 열 추가
- 재적 기간: 연월 칼럼(now_date)과 start_date 칼럼의 차이

In [11]:
predict_data["period"] = 0
predict_data["now_date"] = pd.to_datetime(predict_data["연월"], format="%Y%m")
predict_data["start_date"] = pd.to_datetime(predict_data["start_date"])
for i in range(len(predict_data)):
    delta = relativedelta(predict_data["now_date"][i], predict_data["start_date"][i])
    predict_data["period"][i] = int(delta.years*12 + delta.months)
predict_data.head()

,연월,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date,period,now_date
0,201903,PL666665,7,3.0,XXX,C01,M,2016-12-01,NaT,CA1,...,5.416667,5.0,9.0,3.0,1.0,2019-04-30,28.0,NaN,27,2019-03-01
1,201903,IK287071,3,8.0,XXXXX,C02,F,2017-04-01,NaT,CA1,...,6.333333,7.0,9.0,3.0,1.0,2019-04-30,24.0,NaN,23,2019-03-01
2,201902,HI754520,3,5.0,XXXXX,C01,F,2016-04-01,NaT,CA1,...,4.083333,4.0,6.0,3.0,1.0,2019-04-30,36.0,NaN,34,2019-02-01
3,201812,PL459045,2,5.0,XXXX,C01,M,2016-05-01,NaT,CA1,...,5.000000,5.5,8.0,2.0,1.0,2019-04-30,35.0,NaN,31,2018-12-01
4,201810,AS902294,7,5.0,XXXXXX,C02,F,2017-07-01,NaT,CA2,...,5.833333,6.5,7.0,2.0,1.0,2019-04-30,21.0,NaN,15,2018-10-01


## 45: 결측치 제거

결측치 수 파악

In [12]:
predict_data.isna().sum()

연월                      0
customer_id             0
count_0                 0
count_1               245
name                    0
class                   0
gender                  0
start_date              0
end_date             2842
campaign_id             0
is_deleted              0
class_name              0
price                   0
campaign_name           0
mean                    0
median                  0
max                     0
min                     0
routine_flg             0
calc_date               0
membership_period       0
exit_date            2842
period                  0
now_date                0
dtype: int64

- count_1, end_date, exit_date에 결측치 존재함을 확인
- count_1이 결손된 데이터만 제거

In [13]:
predict_data = predict_data.dropna(subset=["count_1"]) #subset: 특정 칼럼의 결측 데이터 제거

In [14]:
predict_data.isna().sum()

연월                      0
customer_id             0
count_0                 0
count_1                 0
name                    0
class                   0
gender                  0
start_date              0
end_date             2649
campaign_id             0
is_deleted              0
class_name              0
price                   0
campaign_name           0
mean                    0
median                  0
max                     0
min                     0
routine_flg             0
calc_date               0
membership_period       0
exit_date            2649
period                  0
now_date                0
dtype: int64

## 46: 문자열 변수를 처리할 수 있게 가공

- 카데고리 변수: 가입 캠페인 구분, 회원 구분, 성별과 같은 문자열 데이터
    - 활용방안: 플래그 만들기 (더미 변수)

In [16]:
predict_data.head()

,연월,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date,period,now_date
0,201903,PL666665,7,3.0,XXX,C01,M,2016-12-01,NaT,CA1,...,5.416667,5.0,9.0,3.0,1.0,2019-04-30,28.0,NaN,27,2019-03-01
1,201903,IK287071,3,8.0,XXXXX,C02,F,2017-04-01,NaT,CA1,...,6.333333,7.0,9.0,3.0,1.0,2019-04-30,24.0,NaN,23,2019-03-01
2,201902,HI754520,3,5.0,XXXXX,C01,F,2016-04-01,NaT,CA1,...,4.083333,4.0,6.0,3.0,1.0,2019-04-30,36.0,NaN,34,2019-02-01
3,201812,PL459045,2,5.0,XXXX,C01,M,2016-05-01,NaT,CA1,...,5.000000,5.5,8.0,2.0,1.0,2019-04-30,35.0,NaN,31,2018-12-01
4,201810,AS902294,7,5.0,XXXXXX,C02,F,2017-07-01,NaT,CA2,...,5.833333,6.5,7.0,2.0,1.0,2019-04-30,21.0,NaN,15,2018-10-01


In [17]:
target_col = ["campaign_name", "class_name", "gender", "count_1", "routine_flg", "period", "is_deleted"]
predict_data = predict_data[target_col]
predict_data.head()

,campaign_name,class_name,gender,count_1,routine_flg,period,is_deleted
0,일반,종일,M,3.0,1.0,27,0.0
1,일반,주간,F,8.0,1.0,23,0.0
2,일반,종일,F,5.0,1.0,34,0.0
3,일반,종일,M,5.0,1.0,31,0.0
4,입회비반액할인,주간,F,5.0,1.0,15,0.0


카테고리 변수를 이용해서 더미 변수 만들기
- pd.get_dummies(): 범주형 데이터(문자열, 범주형 값 등)를 수치형 값으로 변환
    - 문자열이나 정수형 범주 값을 자동으로 인식하여 각 범주마다 새로운 열 생성


In [19]:
predict_data = pd.get_dummies(predict_data) #문자열 데이터를 칼럼에 저장 -> 더미변수)
predict_data.head()

,count_1,routine_flg,period,is_deleted,campaign_name_일반,campaign_name_입회비무료,campaign_name_입회비반액할인,class_name_야간,class_name_종일,class_name_주간,gender_F,gender_M
0,3.0,1.0,27,0.0,True,False,False,False,True,False,False,True
1,8.0,1.0,23,0.0,True,False,False,False,False,True,True,False
2,5.0,1.0,34,0.0,True,False,False,False,True,False,True,False
3,5.0,1.0,31,0.0,True,False,False,False,True,False,False,True
4,5.0,1.0,15,0.0,False,False,True,False,False,True,True,False


- (주의) 더미 변수별로 겹치는 내용이 있을 수 있기 때문에 이를 처리해줘야함
- campaign_name_일반, class_name_야간, gender_M 삭제

In [20]:
del predict_data["campaign_name_일반"]
del predict_data["class_name_야간"]
del predict_data["gender_M"]
predict_data.head()

,count_1,routine_flg,period,is_deleted,campaign_name_입회비무료,campaign_name_입회비반액할인,class_name_종일,class_name_주간,gender_F
0,3.0,1.0,27,0.0,False,False,True,False,False
1,8.0,1.0,23,0.0,False,False,False,True,True
2,5.0,1.0,34,0.0,False,False,True,False,True
3,5.0,1.0,31,0.0,False,False,True,False,False
4,5.0,1.0,15,0.0,False,True,False,True,True


## 47: 의사결정 트리를 사용하여 탈퇴 예측 모델 구축

In [21]:
from sklearn.tree import DecisionTreeClassifier
import sklearn.model_selection

exit = predict_data.loc[predict_data["is_deleted"]==1]
conti = predict_data.loc[predict_data["is_deleted"]==0].sample(len(exit)) # 샘플 데이터 추출; 두 변수 개수 맞춤

X = pd.concat([exit, conti], ignore_index=True)
y = X["is_deleted"]
del X["is_deleted"]
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X,y)
#학습데이터, 평가데이터로 나누기

model = DecisionTreeClassifier(random_state=0)
model.fit(X_train, y_train)
y_test_pred = model.predict(X_test)
print(y_test_pred)

[1. 0. 1. 0. 1. 1. 0. 1. 0. 0. 1. 1. 1. 0. 1. 0. 0. 1. 1. 1. 0. 1. 0. 1.
 1. 0. 1. 1. 0. 1. 0. 0. 0. 1. 0. 1. 1. 0. 0. 1. 1. 0. 1. 1. 0. 1. 1. 1.
 0. 1. 1. 0. 0. 0. 1. 0. 1. 1. 1. 1. 0. 1. 0. 0. 1. 1. 1. 1. 0. 1. 1. 1.
 0. 1. 0. 0. 0. 1. 1. 1. 0. 1. 1. 1. 1. 0. 0. 0. 0. 1. 1. 0. 1. 1. 0. 0.
 1. 0. 1. 1. 0. 1. 1. 0. 0. 0. 1. 1. 1. 1. 0. 1. 0. 0. 1. 0. 0. 0. 1. 1.
 0. 0. 1. 1. 1. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 0.
 0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 1. 1. 0. 0.
 1. 1. 0. 1. 0. 0. 1. 0. 1. 1. 0. 1. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 1. 1.
 1. 0. 0. 0. 0. 0. 1. 0. 1. 1. 0. 1. 1. 1. 0. 1. 0. 1. 0. 1. 0. 0. 0. 0.
 0. 1. 1. 1. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 1. 1. 0.
 0. 1. 0. 0. 1. 1. 0. 0. 0. 1. 1. 1. 0. 1. 0. 0. 1. 1. 0. 1. 0. 0. 1. 1.
 0. 1. 0. 0. 1. 1. 1. 0. 0. 1. 0. 0. 0. 1. 1. 0. 1. 0. 1. 1. 1. 0. 0. 0.
 1. 0. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 0. 1. 0.
 1. 1. 1. 1. 0. 0. 1. 1. 0. 1. 0. 1. 1. 1. 0. 1. 1.

- 출력결과: 1이 탈퇴, 0이 유지

실제 정답과 비교

In [22]:
results_test = pd.DataFrame({"y_test":y_test ,"y_pred":y_test_pred })
results_test.head()

,y_test,y_pred
280,1.0,1.0
1969,0.0,0.0
458,1.0,1.0
1359,0.0,0.0
689,1.0,1.0


## 48: 예측 모델 평가, 모델 튜닝

results_test 데이터 집계해서 정답률 계산

In [23]:
correct = len(results_test.loc[results_test["y_test"]==results_test["y_pred"]])
data_count = len(results_test)
score_test = correct / data_count
print(score_test)

0.8973384030418251


각각의 데이터를 이용했을 때의 정확도: score 활용

In [25]:
print(model.score(X_test, y_test))
print(model.score(X_train, y_train))

0.8973384030418251
0.976552598225602


- 학습용 데이터에 너무 맞춘 과적합 경향 있음
- 데이터 늘리기, 변수 재검토, 모델의 파리미터 변경과 같은 solution 존재

모델의 파라미터 바꾸기: 트리의 깊이를 얕게 하면 모델을 단순화할 수 있다

In [26]:
X = pd.concat([exit, conti], ignore_index=True)
y = X["is_deleted"]
del X["is_deleted"]
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X,y)

model = DecisionTreeClassifier(random_state=0, max_depth=5) # max_depth=5: 의사결정 트리의 깊이가 5단계에서 멈춤
model.fit(X_train, y_train)
print(model.score(X_test, y_test))
print(model.score(X_train, y_train))

0.9296577946768061
0.926489226869455


- 평가, 학습용 데이터 모두 92% 정도
- 과적합을 낮추고 평가용 데이터의 정답률 높임

## 49: 모델에 기여하는 변수 확인

변수와 중요도를 데이터프레임에 저장

In [28]:
importance = pd.DataFrame({"feature_names":X.columns, "coefficient":model.feature_importances_})
importance

,feature_names,coefficient
0,count_1,0.353602
1,routine_flg,0.126626
2,period,0.518527
3,campaign_name_입회비무료,0.000000
4,campaign_name_입회비반액할인,0.000909
5,class_name_종일,0.000336
6,class_name_주간,0.000000
7,gender_F,0.000000


## 50: 회원 탈퇴 예측

예측을 위한 데이터 작성 (변수 정의)

In [29]:
count_1 = 3
routing_flg = 1
period = 10
campaign_name = "입회비무료"
class_name = "종일"
gender = "M"

데이터 가공

In [30]:
if campaign_name == "입회비반값할인":
    campaign_name_list = [1, 0]
elif campaign_name == "입회비무료":
    campaign_name_list = [0, 1]
elif campaign_name == "일반":
    campaign_name_list = [0, 0]
if class_name == "종일":
    class_name_list = [1, 0]
elif class_name == "주간":
    class_name_list = [0, 1]
elif class_name == "야간":
    class_name_list = [0, 0]
if gender == "F":
    gender_list = [1]
elif gender == "M":
    gender_list = [0]
input_data = [count_1, routing_flg, period]
input_data.extend(campaign_name_list)
input_data.extend(class_name_list)
input_data.extend(gender_list)

In [31]:
print(model.predict([input_data]))
print(model.predict_proba([input_data]))

[1.]
[[0. 1.]]
